# Выгрузка статей

In [22]:
import requests
from bs4 import BeautifulSoup
import re
import pickle

Загрузка кода страницы по дню, месяцу и году.

In [2]:
mainpage = "https://www.vedomosti.ru/archive"
def getPageCode(year,month,day):
    response = requests.get(mainpage+"/"+str(year)+"/"+str(month)+"/"+str(day))
                                       # выгружает данные по ссылке
    html = response.content            # переводит их в читаемый формат, который можно вывести на экран
    soup = BeautifulSoup(html,"lxml")  # отсекает кучу всяких ненужных вещей и запускает поиск по тегам в html  
           # отсекает ошибки, когда в 1 на 10000 статей не читается какой-то символ!
    return soup

Функция, которая осуществляет поиск в заголовках по маркерам. Если маркер в нем присутствует, то функция выдаёт единицу.

In [3]:
m = [' набиулл',' центробанк',' юдаев',' банк россии ',' цб ']

def IsCBNews(news):
    l = 0
    answer = 0
    for item in m:
        l = l + news.count(item)
    if l > 0:
        answer = 1
    return(answer)    

Функция, возвращающая список ссылок и заголовков, ксающихся ЦБ, за конкретное число. 

In [4]:
def getPageUrl(year,month,day):
    df = getPageCode(year,month,day)
    vvv = df.findAll("div", { "class" : "b-article__title" })
    vec = [re.split('>|"|<',str(vvv[item].findAll('a'))) for item in range(len(vvv))]
    url = [[item[2],item[4].lower()] for item in vec if IsCBNews(item[4].lower())==1]
    return url

In [5]:
getPageUrl(2016,7,1)

[['/finance/news/2016/07/01/647604-kostin-olimpiitsev-tsb-stavku',
  'андрей костин просит «олимпийцев» из цб «рубануть» ставку на полпроцента'],
 ['/economics/news/2016/07/01/647593-minfin-i-tsb-podgotovili-predlozheniya-o-dobrovolnih-pensionnih-nakoplenii',
  'минфин и цб подготовили предложения о формировании добровольных пенсионных накоплений'],
 ['/library/news/2016/07/01/647583-degtyarev-kominterna',
  'депутат дегтярев спросил минфин и цб о затратах ссср на поддержку коминтерна']]

Функция, которая выдаёт текст статей, касающихся ЦБ за конкретный день

In [6]:
def text(url):
    response = requests.get('http://www.vedomosti.ru'+url)
    html = response.content 
    soup = BeautifulSoup(html,"lxml")
    dd = soup.findAll("div", { "class" : "b-news-item__text b-news-item__text_one" })
    tx = re.split('</script>',str(dd[0]))[1]
    visible_text = re.sub(u"[^а-яА-Я0-9., ]", "", tx)     #очистка текста от хлама!
    return visible_text

Функция, которая возвращает вообще всё за конкретный день :)

In [18]:
def Dict(year,month,day):
    df = getPageUrl(year,month,day)
    dict = {
        "url":[],
        "title":[],
        "text": [],}
    for item in df:
        dict['url'].append(item[0])
        dict['title'].append(item[1])
        dict['text'].append(text(item[0]))
    return(dict)

Блок, отвечающий за выгрузку статьи на рабочий стол!

In [ ]:
cd "C:\Users\zero\Desktop\mydata"

In [ ]:
# Команды для нормальной выгрузки и загрузки словарей:
# import pickle
# pickle.dump( row, open( "save.p", "wb" )) 
# hh = pickle.load( open( "save.p", "rb" ) )

In [19]:
# Вспомогательная функция для получения правильного количества дней. Работает даже с високосным годом.
def monthlength(month,year):
    if year % 4 == 0:
         VisYear = 29
    else:
         VisYear = 28
    return [31,VisYear,31,30,31,30,31,31,30,31,30,31][month]

In [20]:
# Дни года!
def DaysOfYear(year):
    def Days(month):
        return([i for i in range(1,monthlength(month,2015)+1)])
    s = []
    for i in range(0,12):
        s.append(Days(i))
    return(s)

In [21]:
#Кусок кода для выгрузки!
year = 2016
month = 1
days = DaysOfYear(2015)[month]
len(days)

28

In [ ]:
for number in lll:
    pickle.dump(getCBList(2015,6,number), open(str(number)+'.txt', "wb" )) 

### Разобраться с ошибкой!

In [25]:
d = Dict(2016,7,1)

In [23]:
#Почему-то вылетает ошибка на 17 июня!
Dict(2015,6,17)

IndexError: list index out of range